# Qwen Coder - No Tools Mode

Strips tool definitions to force normal text responses.

In [ ]:
!nvidia-smi
!curl -fsSL https://ollama.com/install.sh | sh
!wget -q https://github.com/cloudflare/cloudflared/releases/latest/download/cloudflared-linux-amd64.deb && dpkg -i cloudflared-linux-amd64.deb
!pip install -q flask requests
print('✅ Setup done')

In [ ]:
import subprocess, time, os
os.environ['OLLAMA_HOST'] = '0.0.0.0:11434'
os.environ['OLLAMA_ORIGINS'] = '*'
subprocess.Popen(['ollama', 'serve'], stdout=subprocess.PIPE, stderr=subprocess.PIPE)
time.sleep(5)
!ollama pull qwen2.5-coder:7b
print('\n✅ Model ready')

In [ ]:
# API that strips tools and forces text responses
from flask import Flask, request, jsonify
import requests as req
import json, time, uuid, threading

app = Flask(__name__)

SYSTEM_PROMPT = """You are a helpful coding assistant. 
IMPORTANT: Always respond with helpful text and code. 
Never output JSON tool calls. 
When asked to create files or code, write out the complete code directly.
Format code with markdown code blocks."""

@app.route('/v1/models', methods=['GET'])
def models():
    return jsonify({"object": "list", "data": [{"id": "qwen2.5-coder:7b", "object": "model", "owned_by": "ollama"}]})

@app.route('/v1/chat/completions', methods=['POST'])
def chat():
    data = request.json
    messages = data.get('messages', [])
    
    # Clean and prepare messages - NO TOOLS
    clean_messages = [{'role': 'system', 'content': SYSTEM_PROMPT}]
    
    for m in messages:
        role = m.get('role', 'user')
        content = m.get('content', '')
        
        # Skip system messages (we add our own)
        if role == 'system':
            continue
        # Convert tool messages to user messages
        if role == 'tool':
            continue
        # Skip if no content
        if not content or not str(content).strip():
            continue
            
        clean_messages.append({'role': role, 'content': str(content)})
    
    # Make sure there's at least one user message
    if len(clean_messages) == 1:
        clean_messages.append({'role': 'user', 'content': 'Hello, can you help me?'})
    
    # Call Ollama - NO TOOLS PASSED
    try:
        r = req.post('http://localhost:11434/api/chat', json={
            'model': 'qwen2.5-coder:7b',
            'messages': clean_messages,
            'stream': False,
            'options': {'num_ctx': 8192}
        }, timeout=120)
        
        content = r.json().get('message', {}).get('content', '')
        if not content:
            content = "I'm ready to help with your coding task. What would you like me to do?"
    except Exception as e:
        content = f"I'm here to help. What code would you like me to write?"
    
    return jsonify({
        "id": f"chatcmpl-{uuid.uuid4().hex[:12]}",
        "object": "chat.completion",
        "created": int(time.time()),
        "model": "qwen2.5-coder:7b",
        "choices": [{"index": 0, "message": {"role": "assistant", "content": content}, "finish_reason": "stop"}],
        "usage": {"prompt_tokens": 50, "completion_tokens": 50, "total_tokens": 100}
    })

threading.Thread(target=lambda: app.run(host='0.0.0.0', port=5000, threaded=True, use_reloader=False), daemon=True).start()
time.sleep(2)
print('✅ No-Tools API running on port 5000')

In [ ]:
# Test
import requests
r = requests.post('http://localhost:5000/v1/chat/completions', json={
    'model': 'qwen2.5-coder:7b',
    'messages': [{'role': 'user', 'content': 'Write a hello world in Python'}],
    'tools': [{'type': 'function', 'function': {'name': 'test'}}]  # Simulating OpenCode sending tools
})
print("Response:", r.json()['choices'][0]['message']['content'][:300])

In [ ]:
# Tunnel
import subprocess, re
from IPython.display import display, HTML

tunnel = subprocess.Popen(['cloudflared', 'tunnel', '--url', 'http://localhost:5000'],
    stdout=subprocess.PIPE, stderr=subprocess.STDOUT, text=True)

for line in tunnel.stdout:
    print(line, end='')
    if 'trycloudflare.com' in line:
        m = re.search(r'https://[^\s]+\.trycloudflare\.com', line)
        if m:
            url = m.group()
            display(HTML(f'''<div style="background:#e74c3c;padding:25px;border-radius:15px">
            <h2 style="color:white">🚫 No-Tools Mode - Text Only</h2>
            <p style="color:white;font-size:18px;font-family:monospace">{url}/v1</p>
            <p style="color:#eee">Tools stripped - model will respond with text/code only</p>
            </div>'''))
            break

for line in tunnel.stdout:
    print(line, end='')